# Fake News Classifier Using Bidirectional LSTM

In [1]:
import pandas as pd

In [2]:
# Read data from csv
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
# Drop Nan values
df=df.dropna()

In [5]:
# Get the Independent features
x=df.drop('label', axis=1)

In [6]:
# Get the Dependent features
y=df['label']

In [7]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [9]:
x.shape,y.shape

((18285, 4), (18285,))

In [10]:
import tensorflow as tf

In [13]:
from tensorflow.keras.layers import Embedding,LSTM,Dense,Bidirectional,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot


In [14]:
# Vocabulary size
voc_size=5000

### Onehot Representation

In [16]:
message=x.copy()

In [17]:
message['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [18]:
message.reset_index(inplace=True)

In [20]:
import nltk
import re
from nltk.corpus import stopwords

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91800\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [23]:
# Datasets Preprocessing
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus = []
for i in range(0, len(message)):
    review = re.sub('[^a-zA-Z]', ' ', message['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [24]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [25]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[4192, 3928, 695, 1580, 231, 1937, 3930, 2771, 3658, 347],
 [4312, 2595, 4468, 2881, 2381, 3381, 2685],
 [1247, 4410, 816, 3700],
 [4217, 2230, 2396, 3022, 4203, 2981],
 [1598, 2381, 1202, 1146, 895, 1590, 2381, 4274, 3242, 109],
 [2979,
  2154,
  4346,
  4770,
  2583,
  2756,
  601,
  705,
  3167,
  4067,
  2814,
  3134,
  4339,
  4251,
  2685],
 [1248, 2300, 1746, 3418, 4500, 3594, 1287, 4797, 4131, 2881, 1536],
 [4346, 1030, 318, 1157, 3731, 2788, 2756, 4224, 4131, 2881, 1536],
 [2805, 2008, 4294, 3108, 1081, 3959, 611, 3499, 2756, 2151],
 [478, 602, 106, 1106, 3971, 3073, 1841, 1749],
 [340, 1180, 3822, 4529, 3805, 1707, 1647, 4443, 4904, 1974, 1457],
 [3022, 2720, 231, 3959, 2756, 3731],
 [3450, 4083, 4264, 4277, 2976, 1505, 353, 1663, 2677],
 [4808, 1720, 1793, 598, 2981, 3804, 20, 4131, 2881, 1536],
 [643, 2644, 334, 1110, 2431, 4131, 2881, 1536],
 [627, 3333, 4837, 2649, 3910, 730, 725, 1053, 3355, 3503],
 [4561, 2762, 2595],
 [2217, 4203, 326, 3751, 2756, 3570, 3643, 2685],
 

### Embedding Representation

In [26]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2771 3658  347]
 [   0    0    0 ... 2381 3381 2685]
 [   0    0    0 ... 4410  816 3700]
 ...
 [   0    0    0 ... 4131 2881 1536]
 [   0    0    0 ... 4102   52 2091]
 [   0    0    0 ... 3491 4014 2128]]


In [28]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4192,
       3928,  695, 1580,  231, 1937, 3930, 2771, 3658,  347])

In [30]:
# creating model 
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
# Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [32]:
len(embedded_docs),y.shape

(18285, (18285,))

In [33]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [34]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [36]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 12s 61ms/step - loss: 0.3162 - accuracy: 0.8524 - val_loss: 0.2036 - val_accuracy: 0.9158
Epoch 2/10
192/192 [==============================] - 8s 43ms/step - loss: 0.1406 - accuracy: 0.9465 - val_loss: 0.1959 - val_accuracy: 0.9196
Epoch 3/10
192/192 [==============================] - 8s 44ms/step - loss: 0.0951 - accuracy: 0.9658 - val_loss: 0.2076 - val_accuracy: 0.9210
Epoch 4/10
192/192 [==============================] - 8s 44ms/step - loss: 0.0664 - accuracy: 0.9776 - val_loss: 0.2466 - val_accuracy: 0.9188
Epoch 5/10
192/192 [==============================] - 9s 45ms/step - loss: 0.0431 - accuracy: 0.9860 - val_loss: 0.3197 - val_accuracy: 0.9128
Epoch 6/10
192/192 [==============================] - 8s 43ms/step - loss: 0.0266 - accuracy: 0.9911 - val_loss: 0.3540 - val_accuracy: 0.9130
Epoch 7/10
192/192 [==============================] - 8s 43ms/step - loss: 0.0151 - accuracy: 0.9951 - val_loss: 0.4180 - val_accuracy: 0.909

### Performance Metrics And Accuracy

In [37]:
y_pred1=model1.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [38]:
from sklearn.metrics import confusion_matrix

In [40]:
confusion_matrix(y_test,y_pred1)

array([[3098,  321],
       [ 211, 2405]], dtype=int64)

In [41]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.911847555923778

In [42]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.94      0.91      0.92      3419
           1       0.88      0.92      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

